In [ ]:
import os, re, math, json, shutil, pprint
from tensorflow.keras import backend as K
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import IPython.display as display
from tensorflow.keras.utils import to_categorical

import numpy as np
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import losses, optimizers
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Flatten
import pandas as pd
print("Tensorflow version " + tf.__version__)
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Data loading

In [ ]:
BATCH_SIZE = 128

In [ ]:
#@title visualization utilities [RUN ME]
"""
This cell contains helper functions used for visualization
and downloads only. You can skip reading it. There is very
little useful Keras/Tensorflow code here.
"""



  

def plot_learning_rate(lr_func, epochs):
  xx = np.arange(epochs+1, dtype=np.float)
  y = [lr_decay(x) for x in xx]
  fig, ax = plt.subplots(figsize=(9, 6))
  ax.set_xlabel('epochs')
  ax.set_title('Learning rate\ndecays from {:0.3g} to {:0.3g}'.format(y[0], y[-2]))
  ax.minorticks_on()
  ax.grid(True, which='major', axis='both', linestyle='-', linewidth=1)
  ax.grid(True, which='minor', axis='both', linestyle=':', linewidth=0.5)
  ax.step(xx,y, linewidth=3, where='post')
  display.display(fig)

class PlotTraining(tf.keras.callbacks.Callback):
  def __init__(self, sample_rate=1, zoom=1):
    self.sample_rate = sample_rate
    self.step = 0
    self.zoom = zoom
    self.steps_per_epoch = 60000//BATCH_SIZE

  def on_train_begin(self, logs={}):
    self.batch_history = {}
    self.batch_step = []
    self.epoch_history = {}
    self.epoch_step = []
    self.fig, self.axes = plt.subplots(1, 2, figsize=(16, 7))
    plt.ioff()

  def on_batch_end(self, batch, logs={}):
    if (batch % self.sample_rate) == 0:
      self.batch_step.append(self.step)
      for k,v in logs.items():
        # do not log "batch" and "size" metrics that do not change
        # do not log training accuracy "acc"
        if k=='batch' or k=='size':# or k=='acc':
          continue
        self.batch_history.setdefault(k, []).append(v)
    self.step += 1

  def on_epoch_end(self, epoch, logs={}):
    plt.close(self.fig)
    self.axes[0].cla()
    self.axes[1].cla()
      
    self.axes[0].set_ylim(0, 1.2/self.zoom)
    self.axes[1].set_ylim(1-1/self.zoom/2, 1+0.1/self.zoom/2)
    
    self.epoch_step.append(self.step)
    for k,v in logs.items():
      # only log validation metrics
      if not k.startswith('val_'):
        continue
      self.epoch_history.setdefault(k, []).append(v)

    display.clear_output(wait=True)
    
    for k,v in self.batch_history.items():
      self.axes[0 if k.endswith('loss') else 1].plot(np.array(self.batch_step) / self.steps_per_epoch, v, label=k)
      
    for k,v in self.epoch_history.items():
      self.axes[0 if k.endswith('loss') else 1].plot(np.array(self.epoch_step) / self.steps_per_epoch, v, label=k, linewidth=3)
      
    self.axes[0].legend()
    self.axes[1].legend()
    self.axes[0].set_xlabel('epochs')
    self.axes[1].set_xlabel('epochs')
    self.axes[0].minorticks_on()
    self.axes[0].grid(True, which='major', axis='both', linestyle='-', linewidth=1)
    self.axes[0].grid(True, which='minor', axis='both', linestyle=':', linewidth=0.5)
    self.axes[1].minorticks_on()
    self.axes[1].grid(True, which='major', axis='both', linestyle='-', linewidth=1)
    self.axes[1].grid(True, which='minor', axis='both', linestyle=':', linewidth=0.5)
    display.display(self.fig)

In [ ]:
df_images = pd.read_csv('/content/drive/MyDrive/dl/data/age_gender.csv')

In [ ]:
df_images.sample(5)

In [ ]:
df_images.dtypes

In [ ]:
## delete img_name
# df_images = df_images.drop('img_name', axis=1)
## 
df_images["age"] = df_images["age"].apply(lambda x: "enfant" if x < 18 else("jeune" if 18 <= x < 30 else( "Adultes" if 30 <= x < 55 else "vieux")))
from sklearn.preprocessing import LabelEncoder
df_age = df_images[['age', 'pixels']]
Encoder_Function = LabelEncoder()
df_age["age"] = Encoder_Function.fit_transform(df_age["age"])
# print(df_age["age"].value_counts())
df_gender = df_images[['gender', 'pixels']]


In [ ]:
# df_age.sample(4)
df_gender.sample(4)

In [ ]:
df_gender['gender'].value_counts()

In [ ]:
len(set(df_age['age'].tolist()))

In [ ]:
df_gender["pixels"] = df_gender["pixels"].apply(lambda x: np.array(x.split(),dtype="float32"))
df_gender["pixels"] = df_gender["pixels"] / 255

In [ ]:
df_gender.sample(4)

In [ ]:
pixels = np.zeros(shape=(23705,2304))
for i in range(len(df_gender["pixels"])):
    pixels[i] = df_gender["pixels"][i]


In [ ]:
pixels.shape

In [ ]:
df_age["age"] = np.array(df_age["age"],dtype="float32")

In [ ]:
Y_Train = df_gender['gender']
#train et test split
x_train, x_test, y_train, y_test = train_test_split(pixels,Y_Train , test_size=0.33, random_state=42)
x_train = x_train.reshape(15882, 2304)
x_test = x_test.reshape(7823, 2304)

In [ ]:
# X_Train.shape
print(x_train.shape, x_test.shape)

In [ ]:
plt.figure(figsize=(10, 4))
for i in range(8):
    plt.subplot(1, 8, i+1)
    plt.imshow(pixels[i+6000].reshape(48,48), 
               interpolation="none")
    plt.title(Y_Train[i], fontsize=14)
    plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
# convert class vectors to binary class matrices
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
y_train[0].shape

### premier modèle

In [ ]:
inputs = tf.keras.Input(shape=(2304,), name="faces")
outputs = tf.keras.layers.Dense(3, activation="softmax", name="predictions")(inputs)

model = tf.keras.Model(inputs=inputs, outputs=outputs)




model.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# print model layers
model.summary()

# #utility callback that displays training curves
plot_training = PlotTraining(sample_rate=10, zoom=1)

In [ ]:
model2 = tf.keras.Sequential(
  [
      tf.keras.layers.Reshape(input_shape=(2304,), target_shape=(48, 48, 1), name='conv2d'),
      Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'),
      BatchNormalization(),
      Dropout(0.25, name='dropout_'),

      Conv2D(32, kernel_size=(3, 3), activation='relu', name='conv2d__'),
      BatchNormalization(),
      MaxPooling2D(pool_size=(2, 2), name='max_pool_'),
      Dropout(0.25, name='dropout__'),

      Conv2D(64, kernel_size=(3, 3), activation='relu', name='conv2d___'),
      BatchNormalization(),
      Dropout(0.25, name='dropout___'),

      Conv2D(64, kernel_size=(3, 3), activation='relu', name='conv2d____'),
      BatchNormalization(),
      MaxPooling2D(pool_size=(2, 2), name='max_pool__'),
      Dropout(0.25, name='dropout____'),

      Flatten(name='flatte'),
      Dense(512, activation='relu', name='dense_'),
      BatchNormalization(),
      Dropout(0.5, name='dropout_____'),
      Dense(128, activation='relu', name='dense__'),
      BatchNormalization(),
      Dropout(0.5, name='dropout______'),
      Dense(2, activation='softmax', name='out'),
  ])


In [ ]:
model2.history

In [ ]:
model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



# print model layers
model2.summary()
#afficher les courbes
plot_training = PlotTraining(sample_rate=10, zoom=1)
# compiler le modele
model2.compile(loss=losses.categorical_crossentropy,
                  optimizer=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
                  metrics=['accuracy'])
model2.summary()

In [ ]:
tf.config.experimental_run_functions_eagerly(True)
print("Fit model on training data")
history = model2.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=15,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_test, y_test),
    validation_steps=1,
    
    callbacks=[plot_training],

    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
)


In [ ]:
history.history

In [ ]:
model2.evaluate(x_test,y_test)

In [ ]:
!pip install h5py

In [ ]:
model2.save('model2.h5')

In [ ]:
model2_save = tf.keras.models.load_model('/content/drive/MyDrive/model2.h5')

In [ ]:
model2_save.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
df_hist= pd.DataFrame(history.history)

df_hist.plot()

In [ ]:
model2_save.summary()

In [ ]:
import numpy as np
import cv2

img = cv2.imread('/content/drive/MyDrive/Filip-Nikolic-2Be3-decouvrez-Sasha-sa-fille-de-15-ans-deja-star-d-Instagram.jpg')
pic = cv2.resize(img,(48,48))
pic_ = cv2.cvtColor(pic, cv2.COLOR_BGR2GRAY)
pic_.shape
#pic_= np.reshape(pic,(48,48))
pic_= np.reshape(pic_,(1,2304))

#pic_

In [ ]:
pic_ = pic_  / 255
pic_

In [ ]:
picture_predict = np.argmax(model2_save.predict(pic_),axis=1)
print(picture_predict)

In [ ]:
model2_save.predict(pic_)

In [ ]:
#img = cv2.imread('/content/drive/MyDrive/tetu-28-04-2017-laissez-vous-seduire-garcon-francais-4.jpg')
#pic = np.resize(img,(48,48))
#pic_= np.reshape(pic,(48,48))
#pic_= np.reshape(pic_,(1,2304))

#pic_
#pic_= pic_ / 255
#pic_
#model2_save.predict(pic_)

# **ethnicity**

In [ ]:
df_ethnicity = df_images[['ethnicity', 'pixels']]

In [ ]:
df_ethnicity["pixels"] = df_ethnicity["pixels"].apply(lambda x: np.array(x.split(),dtype="float32"))
df_ethnicity["pixels"] = df_ethnicity["pixels"] / 255

In [ ]:
pixels_e = np.zeros(shape=(23705,2304))
for i in range(len(df_ethnicity["pixels"])):
    pixels_e[i] = df_ethnicity["pixels"][i]

In [ ]:
Y_Train = df_ethnicity['ethnicity']
#train et test split
x_train, x_test, y_train, y_test = train_test_split(pixels,Y_Train , test_size=0.33, random_state=42)
x_train = x_train.reshape(15882, 2304)
x_test = x_test.reshape(7823, 2304)

In [ ]:
plt.figure(figsize=(10, 4))
for i in range(8):
    plt.subplot(1, 8, i+1)
    plt.imshow(pixels_e[i].reshape(48,48), 
               interpolation="none")
    plt.title(Y_Train[i], fontsize=14)
    plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
# convert class vectors to binary class matrices
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
inputs = tf.keras.Input(shape=(2304,), name="faces")
outputs = tf.keras.layers.Dense(3, activation="softmax", name="predictions")(inputs)

model_e = tf.keras.Model(inputs=inputs, outputs=outputs)




model_e.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# print model layers
model_e.summary()

# #utility callback that displays training curves
plot_training = PlotTraining(sample_rate=10, zoom=1)

In [ ]:
model2_e = tf.keras.Sequential(
  [
      tf.keras.layers.Reshape(input_shape=(2304,), target_shape=(48, 48, 1), name='conv2d'),
      Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'),
      BatchNormalization(),
      Dropout(0.25, name='dropout_'),

      Conv2D(32, kernel_size=(3, 3), activation='relu', name='conv2d__'),
      BatchNormalization(),
      MaxPooling2D(pool_size=(2, 2), name='max_pool_'),
      Dropout(0.25, name='dropout__'),

      Conv2D(64, kernel_size=(3, 3), activation='relu', name='conv2d___'),
      BatchNormalization(),
      Dropout(0.25, name='dropout___'),

      Conv2D(64, kernel_size=(3, 3), activation='relu', name='conv2d____'),
      BatchNormalization(),
      MaxPooling2D(pool_size=(2, 2), name='max_pool__'),
      Dropout(0.25, name='dropout____'),

      Flatten(name='flatte'),
      Dense(512, activation='relu', name='dense_'),
      BatchNormalization(),
      Dropout(0.5, name='dropout_____'),
      Dense(128, activation='relu', name='dense__'),
      BatchNormalization(),
      Dropout(0.5, name='dropout______'),
      Dense(5, activation='softmax', name='out'),
  ])


In [ ]:
model2_e.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



# print model layers
model2_e.summary()
#afficher les courbes
plot_training = PlotTraining(sample_rate=10, zoom=1)
# compiler le modele
model2_e.compile(loss=losses.categorical_crossentropy,
                  optimizer=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
                  metrics=['accuracy'])
model2_e.summary()

In [ ]:
tf.config.experimental_run_functions_eagerly(True)
print("Fit model on training data")
history = model2_e.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=50,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_test, y_test),
    validation_steps=1,
    
    callbacks=[plot_training],

    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
)

In [ ]:
#model2_e.save('model2_e.h5')
model2_e_save = tf.keras.models.load_model('/content/drive/MyDrive/model2_a.h5')
model2_e_save.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
import numpy as np
import cv2

img = cv2.imread('/content/drive/MyDrive/homme-medecine-chirurgie-esthetique-dr-benouaiche-paris.jpg')
pic = np.resize(img,(48,48))
pic_= np.reshape(pic,(48,48))
pic_= np.reshape(pic_,(1,2304))

pic_

In [ ]:
pic_ = pic_  / 255
pic_

In [ ]:
picture_predict = np.argmax(model2_e_save.predict(pic_),axis=1)
print(picture_predict)

In [ ]:
model2_e_save.predict(pic_)

# **age**

In [ ]:
df_age["pixels"] = df_age["pixels"].apply(lambda x: np.array(x.split(),dtype="float32"))
df_age["pixels"] = df_age["pixels"] / 255

In [ ]:
pixels_a = np.zeros(shape=(23705,2304))
for i in range(len(df_ethnicity["pixels"])):
    pixels_a[i] = df_ethnicity["pixels"][i]

In [ ]:
Y_Train = df_age['age']
#train et test split
x_train, x_test, y_train, y_test = train_test_split(pixels,Y_Train , test_size=0.33, random_state=42)
x_train = x_train.reshape(15882, 2304)
x_test = x_test.reshape(7823, 2304)

In [ ]:
# convert class vectors to binary class matrices
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
inputs = tf.keras.Input(shape=(2304,), name="faces")
outputs = tf.keras.layers.Dense(3, activation="softmax", name="predictions")(inputs)

model_a = tf.keras.Model(inputs=inputs, outputs=outputs)




model_a.compile(optimizer='sgd',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# print model layers
model_a.summary()

# #utility callback that displays training curves
plot_training = PlotTraining(sample_rate=10, zoom=1)

In [ ]:
model2_a = tf.keras.Sequential(
  [
      tf.keras.layers.Reshape(input_shape=(2304,), target_shape=(48, 48, 1), name='conv2d'),
      Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'),
      BatchNormalization(),
      Dropout(0.25, name='dropout_'),

      Conv2D(32, kernel_size=(3, 3), activation='relu', name='conv2d__'),
      BatchNormalization(),
      MaxPooling2D(pool_size=(2, 2), name='max_pool_'),
      Dropout(0.25, name='dropout__'),

      Conv2D(64, kernel_size=(3, 3), activation='relu', name='conv2d___'),
      BatchNormalization(),
      Dropout(0.25, name='dropout___'),

      Conv2D(64, kernel_size=(3, 3), activation='relu', name='conv2d____'),
      BatchNormalization(),
      MaxPooling2D(pool_size=(2, 2), name='max_pool__'),
      Dropout(0.25, name='dropout____'),

      Flatten(name='flatte'),
      Dense(512, activation='relu', name='dense_'),
      BatchNormalization(),
      Dropout(0.5, name='dropout_____'),
      Dense(128, activation='relu', name='dense__'),
      BatchNormalization(),
      Dropout(0.5, name='dropout______'),
      Dense(4, activation='softmax', name='out'),
  ])


In [ ]:
model2_a.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



# print model layers
model2_a.summary()
#afficher les courbes
plot_training = PlotTraining(sample_rate=10, zoom=1)
# compiler le modele
model2_a.compile(loss=losses.categorical_crossentropy,
                  optimizer=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
                  metrics=['accuracy'])
model2_a.summary()

In [ ]:
tf.config.experimental_run_functions_eagerly(True)
print("Fit model on training data")
history = model2_a.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=50,
    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
    validation_data=(x_test, y_test),
    validation_steps=1,
    
    callbacks=[plot_training],

    # We pass some validation for
    # monitoring validation loss and metrics
    # at the end of each epoch
)

In [ ]:
model2_a.save('model2_a.h5')
model2_a_save = tf.keras.models.load_model('model2_a.h5')
model2_a_save.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])